In [1]:
import os
os.chdir('..')

# Import Files

In [2]:
from src.agent_interface import *
from src.agent import *
from src.env import *
from src.price_data import *
from src.prior_measure import *
from src.q import *
from src.robust import *

# Initialize Agent

In [3]:
device = torch.device("cuda")

#Shared params
n_updates = 1
state_dim = 63
action_dim = 1

training_controller_params = {
    'train_steps': 1,
    'clone_steps': 50,
    'batch_size': 128,
    'n_batches': n_updates,
}

duality_params = {
    'discount_rate': 0.99,
    'delta': 1e-4,
    'sinkhorn_radius': 0.003,
}

q_params = {
    'input_size': state_dim,
    'hidden_size': [64, 64],
    'output_size': action_dim
}

dqn_params = {
    'state_dim': state_dim,
    'action_dim': action_dim,
    'batch_size': 128,
    'n_updates': 10,
    'epsilon': 0.1,
    'device': device
}

In [4]:
training_controller = TrainingController(**training_controller_params)
prior_measure = PriorStudentDistribution(device=device)
duality_operator = DualityHQOperator(**duality_params)
q = QFunc(**q_params)
agent = PORDQN(training_controller=training_controller, prior_measure=prior_measure, duality_operator=duality_operator,
               qfunc = q, **dqn_params)